In [14]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import logging as log

log.basicConfig(level=log.DEBUG)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifi

In [2]:
import sys
sys.path.append('../')
from baselines.utils import *
import os

In [3]:
train_data = load_data(os.path.join(cfg['DIR'], cfg['train_path']))
dev_data = load_data(os.path.join(cfg['DIR'], cfg['dev_path']))
test_data = load_data(os.path.join(cfg['DIR'], cfg['test_path']))

In [4]:
hypothesis = get_hypothesis(train_data)

In [5]:
from icecream import ic

In [89]:
from torch.utils.data import Dataset
import torch

class NLIDataset(Dataset):
    def __init__(self, data, tokenizer, hypothesis, max_len = 475):
        self.data = data
        self.tokenizer = tokenizer
        self.hypothesis = hypothesis
        self.max_len = max_len
        self.spans = []
        self.label_dict = get_labels()

        for i, doc in enumerate(self.data['documents']):
            for j, span in enumerate(doc['spans']):
                self.spans.append({
                    'doc_id': i,
                    'span_id': j,
                    'text': doc['text'][span[0]:span[1]]
                })

        self.data_points = []
        self.span_label = []
        self.nli_label = []

        for key, val in self.hypothesis.items():
            for span in self.spans:
                self.span_label.append(int(span['span_id'] in self.data['documents'][span['doc_id']]['annotation_sets'][0]['annotations'][key]['spans']))
                self.data_points.append({
                    'hypotheis': val,
                    'premise': span['text']
                })

                self.nli_label.append(self.label_dict[self.data['documents'][span['doc_id']]['annotation_sets'][0]['annotations'][key]['choice']])

                if self.nli_label[-1] != self.label_dict['NotMentioned'] and self.span_label[-1] == 0:
                    self.nli_label[-1] = self.label_dict['Ignore']


        self.tokenized_data = self.tokenizer(
            [data_point['hypotheis'] for data_point in self.data_points],
            [data_point['premise'] for data_point in self.data_points],
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        self.sep_indices = torch.where(self.tokenized_data['input_ids'] == self.tokenizer.sep_token_id)[1]

        self.sep_indices = self.sep_indices[::2]

    def __len__(self):
        return len(self.data_points)

    def __getitem__(self, idx):
        return \
            self.tokenized_data['input_ids'][idx], \
            self.tokenized_data['attention_mask'][idx], \
            self.tokenized_data['token_type_ids'][idx], \
            torch.tensor(self.span_label[idx], dtype=torch.long), \
            torch.tensor(self.nli_label[idx], dtype=torch.long), \
            self.sep_indices[idx]

In [90]:
# text = "I love Adelaide!"
# # add_special_tokens=True is set by default
# text_enc = tokenizer.encode(text, text, padding='max_length', return_tensors='pt')

# log.debug(text_enc.shape)

# for tok in text_enc:
#     print(tok, tokenizer.decode(tok))

In [91]:
train_dataset = NLIDataset(train_data, tokenizer, hypothesis)
dev_dataset = NLIDataset(dev_data, tokenizer, hypothesis)
test_dataset = NLIDataset(test_data, tokenizer, hypothesis)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
ic(DEVICE)

ic| DEVICE: device(type='cuda')


device(type='cuda')

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
from torch import nn
import torch.nn.functional as F

class ContractNLI(nn.Module):
    def __init__(self, bert, num_labels=4):
        super().__init__()
        self.bert = bert
        self.embedding_dim = self.bert.config.hidden_size
        for param in self.bert.parameters():
            param.requires_grad = False
        self.num_labels = num_labels

        self.span_classifier = nn.Linear(self.embedding_dim, 2)
        self.nli_classifier = nn.Linear(self.embedding_dim, self.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids, sep_index):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, output_hidden_states=True)

        ic(len(outputs))

        sequence_output = outputs.hidden_states[-1]
        ic(sequence_output.shape)

        ic(sep_index.shape)
        ic(sep_index)
        ic(sequence_output[:, sep_index, :].shape)

        span_logits = self.span_classifier(torch.gather(sequence_output, 1, sep_index.unsqueeze(1).expand(-1, sequence_output.shape[-1]).unsqueeze(1)).squeeze(1))
        ic(span_logits.shape)

        nli_logits = self.nli_classifier(sequence_output[:, 0, :])
        ic(nli_logits.shape)

        return span_logits, nli_logits

    def fit(self, train_loader, dev_loader, epochs, lr, ignore_index):
        ic.disable()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        self.nli_criterion = nn.CrossEntropyLoss(ignore_index=ignore_index)
        self.span_criterion = nn.CrossEntropyLoss()
        
        for epoch in range(epochs):
            print(f'Epoch: {epoch + 1}/{epochs}')
            train_loss = self.__train(train_loader)
            print(f'Train Loss: {train_loss:.4f}')
            dev_loss = self.__validate(dev_loader)
            print(f'Dev Loss: {dev_loss:.4f}')

        ic.enable()

    def __train(self, train_loader):
        self.train()
        total_loss = []

        pbar = tqdm(train_loader)

        for batch in pbar:
            loss = self.__call(batch)
            total_loss.append(loss.item())

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            pbar.set_description(f'Loss: {loss.item():.4f}, Average Loss: {np.mean(total_loss):.4f}')

        return np.mean(total_loss)
    
    def __validate(self, dev_loader):
        self.eval()
        total_loss = []

        with torch.no_grad():
            pbar = tqdm(dev_loader)

            for batch in pbar:
                loss = self.__call(batch)
                total_loss.append(loss.item())

                pbar.set_description(f'Loss: {loss.item():.4f}, Average Loss: {np.mean(total_loss):.4f}')

        return np.mean(total_loss)
        
    def __call(self, batch):
        input_ids, attention_mask, token_type_ids, span_label, nli_label, sep_index = tuple(t.to(DEVICE) for t in batch)
        ic(input_ids.shape)
        ic(attention_mask.shape)
        ic(token_type_ids.shape)

        span_logits, nli_logits = self(input_ids, attention_mask, token_type_ids, sep_index)

        span_loss = self.span_criterion(span_logits, span_label)
        nli_loss = self.nli_criterion(nli_logits, nli_label)

        loss = span_loss + nli_loss

        return loss


In [85]:
model = ContractNLI(bert, num_labels=4).to(DEVICE)

In [86]:
model.fit(train_loader, dev_loader, epochs=5, lr=1e-3, ignore_index=get_labels()['Ignore'])

Epoch: 1/5


  0%|          | 0/1 [00:00<?, ?it/s]ic| input_ids.shape: torch.Size([2, 512])
ic| attention_mask.shape: torch.Size([2, 512])
ic| token_type_ids.shape: torch.Size([2, 512])
  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'item'